In [1]:
mes = "07"

In [2]:
# Manipulação e Tratamento de dados
import pandas as pd
import openpyxl
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
####################################-BIBLIOTECAS-####################################

#manipulação de dados
import pandas as pd
import numpy as np

#string e data
import unidecode #limpeza
import re #regex

#tempo
import time
from time import sleep

#web scraping
import requests
from bs4 import BeautifulSoup

#automação
from selenium import webdriver #navegador
from selenium.webdriver.common.keys import Keys #comandos do teclado
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait #esperar
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
import pyautogui #

import warnings #Warning inuteis
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#manipulação do diretorio
import os

#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [4]:
# FUNÇÕES XPATH
def xpath_element(xpath):
    return browser.find_element("xpath", xpath)

def xpath_click(xpath):
    xpath_element(xpath).click()
    
def xpath_send_keys(xpath, texto):
    xpath_element(xpath).send_keys(texto)
    
def xpath_clear(xpath):
    xpath_element(xpath).clear()
    
#####################################################

#FUNÇÕES
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)

def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

# Automação

In [5]:
browser = webdriver.Firefox()

browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
xpath_send_keys('//*[@id="MainContent_LoginUser_UserName"]', 'gabriel_s_anjos@yahoo.com') #e-mail
xpath_send_keys('//*[@id="MainContent_LoginUser_Password"]', 'Gabriel2022!') #senha
xpath_click('//*[@id="MainContent_LoginUser_btnLogin"]') #login

In [6]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [ ]:
#click setor/profissional
xpath_click('//*[@id="s2id_autogen1"]/a/span[1]')
#click [profissional]
xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')

# Capturando setores:

In [ ]:
#click criar novo plantão
def criar_plantao():
    xpath_click('//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button')
    
criar_plantao() #abrindo para capturar os setores

In [ ]:
#CAPTURANDO DADOS DO UNIDADE E SETOR
xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
setor_html = xpath_element('//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1

        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

In [ ]:
sleep(2)
print(pyautogui.position())

In [ ]:
def existencia(caminho):
    elementos = browser.find_elements(By.XPATH, caminho)
    return bool(elementos)

In [ ]:
#função que click em um lugar vazio para fechar caixa de selecão
def reset_selecao():
    posicao_antiga = pyautogui.position() #captura posição atual
    pyautogui.click(x=278, y=169) #click no vazio
    sleep(.3)
    pyautogui.click(x=278, y=169) #click no vazio
    pyautogui.moveTo(posicao_antiga) #voltando o mouse para posição antiga
    
reset_selecao()

In [ ]:
#selecionando setor
def setor(unidade, setor):
    #print('base:', unidade, '-', setor)
    xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
    ind = scrap_unidades.loc[(scrap_unidades.unidade == unidade) & 
                             (scrap_unidades.momento == setor)].index.values[0]
    unidade_sel = scrap_unidades['unidade_count'][ind]
    setor_sel = scrap_unidades['plt_count'][ind]
    xpath_click('//*[@id="select2-drop"]/ul/li['+str(unidade_sel)+']/ul/li['+str(setor_sel)+']/div')
    
setor("SANTA ISABEL", "SEMANA - TARDE - C.CIRURGICO")

#inserir e selecionar profissional
def nome(nome_profissional):
    #listar todos
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    sleep(1)
    xpath_click('//*[@id="s2id_ddlProfessionalA"]/a/span[1]')
    xpath_send_keys('//*[@id="select2-drop"]/div/input', nome_profissional)
    #Dando enter
    xpath_send_keys('//*[@id="select2-drop"]/div/input', Keys.RETURN)
    
nome('OTTO MITTERMAYER')

#inserir horario e duração
def hora(inicio, duracao):
    #Inicio do plantão
    xpath_clear('//*[@id="txtServiceStartTime"]')
    xpath_send_keys('//*[@id="txtServiceStartTime"]', inicio)
    sleep(1)
    xpath_clear('//*[@id="txtServiceDuration"]')
    xpath_send_keys('//*[@id="txtServiceDuration"]', duracao)
    
hora('13:30', '05:00')
    
#inserir data
def data(data_selecionada):
    i_chato = str(3) #2 e 3
    i_chato2 = str(5) #4 e 5
    xpath_clear('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]')
    sleep(1)
    xpath_send_keys('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]', data_selecionada)
    sleep(1)
    
data("10/01/2024")
reset_selecao()

In [ ]:
#inserindo pontuação
def pontuacao(pontos):
    xpath_clear('//*[@id="txtServiceValue"]')
    xpath_send_keys('//*[@id="txtServiceValue"]', pontos) #Pontos

pontuacao("666")
'''
//*[@id="select2-drop"]/ul/li[3]/div
1 - Normal
2 - DESLOCAMENTO
3 - VOLUNTÁRIO
4 - COBERTURA
'''

#selecionar tipo de plantão e ajustando tempo
def tipo_do_registro(duracao, sem_ou_fds, tipo_registro):
    if tipo_registro == 'extra':
        print('EXTRA')
        sleep(1)
        reset_selecao()
        
        xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        xpath_click('//*[@id="select2-drop"]/ul/li[3]/div')
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(200)
            pontuacao(200)
            print('duração >= 5 => 200 pontos')
        else:
            pass
    elif tipo_registro == 'DESLOCADO':
        print('DESLOCADO')
        sleep(1)
        reset_selecao()
        
        xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)
    else:
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)

#tipo_do_registro("05:00", "semana", "extra")
#tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])
        
#salvando plantão
def salvar_plantao():
    xpath_click('//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]')
    
#salvar_plantao()

#fechando plantão
def fechar():
    xpath_click('//*[@id="modal-add-service"]/div/div[1]/button')

fechar()

In [ ]:
def esperar_elemento_aparecer(elemento, tempo):
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))

# CAPTURANDO NOMES

In [ ]:
def xpath_estranho(i):
    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]
    
    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar

xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
xpath_estranho(i)
xpath_click('//*[@id="select2-drop"]/ul/li/div')

In [ ]:
xpath_click('//*[@id="s2id_ddlProfessionals"]/a') #barra de nome

html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML') #
soup_nome = BeautifulSoup(html, 'html.parser')
#soup_nome

NOMES = []
for i in soup_nome.find_all(class_ ="select2-results-dept-0 select2-result select2-result-selectable"):
    #print(i.get_text())
    NOMES.append(i.get_text())
#del(NOMES[0])

reset_selecao()

NOMES

# FUNÇÕES DE CAPTURA DOS DADOS LAYOUT

In [ ]:
def diferenca_horas(horario_inicial, horario_final):
    formato = "%H:%M"
    horario_inicial = datetime.strptime(horario_inicial, formato)
    horario_final = datetime.strptime(horario_final, formato)
    diferenca = horario_final - horario_inicial
    if diferenca.days < 0:
        diferenca = horario_inicial - horario_final
        return diferenca.total_seconds() / 60 * (-1)
    else:
        return diferenca.total_seconds() / 60

def atualizando_plantao():
    xpath_click('/html/body/div[5]/div/div[1]/div[2]/button[5]')
    #xpath_click('/html/body/div[4]/div/div[1]/div[2]/button[5]')
#atualizando_plantao()

In [ ]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 
#capturando registros
unidades = ['ADMINISTRACAO','ALPHA','ANALIA','AVICCENA',
            'ITAIM','JABAQUARA','MATERNIDADE','MORUMBI',
            'OSASCO','VILLA','VNS','SANTA ISABEL']

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando resido do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2024" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    
    return tabela
    
tabela = funcao()
tabela

In [ ]:
# FUNÇÃO ESPERAR APARECER
def esperar_elemento_aparecer(elemento, tempo):
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))

def apagar_registro(ind):
    #print('PRINCIPAL')
    #abrindo/selecionando periodo conforme localização
    xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][ind])+']/td['+str(dia_sel['Dia Semana'][ind])+']/ul['+str(dia_sel['ordem'][ind])+']/li'
    xpath_click(xpath_textao)

    #Esperar componentes carregarem
    WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))

    #Apagando
    esperar_elemento_aparecer('/html/body/div[5]/div/div[1]/div[2]/button[1]', 10)
    xpath_click('/html/body/div[5]/div/div[1]/div[2]/button[1]')
    #Clicando em continuar
    sleep(0.5)
    xpath_click('/html/body/div[5]/div/div[1]/div[3]/button[2]')

    #esperando carregamento do site para prosseguir
    WebDriverWait(browser, 10).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))

#apagar_registro(16)

# INDO PARA CIMA

In [ ]:
count = 0
for NOME in NOMES:
    print(count, '-', NOME)
    count += 1

In [ ]:
count_control = 0
lista = []

In [ ]:
reset_selecao()
count_nome = 0

for NOME in NOMES[count_control:]:

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            count += 1

    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    
    tabela = funcao()
    count = 0
    
    ####################################################################
    #quando o registro for do periodo manhã(2), com saida != 13h => apagar os registros posteriores (tader, noite)
    condicao1 = tabela['periodo'] == 2
    condicao2 = tabela['Saida'] != '13:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]): 
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[tabela['data'] == tabela['data'][i]]
                if len(dia_sel[dia_sel['periodo'] > 2]) > 0:
                    print(tabela['data'][i])
                    print('saida manhã != 13h', '\n')
                    sleep(2)##
                    for ind in dia_sel[dia_sel['periodo'] > 2].index.values: 
                        apagar_registro(ind)
                        
    ####################################################################
    #quando o registro for do periodo tarde(3), com entrada != 13h => apagar os registros anteriores (manha)
    condicao1 = tabela['periodo'] == 3
    condicao2 = tabela['Entrada'] != '13:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]):
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[tabela['data'] == tabela['data'][i]]
                if len(dia_sel[dia_sel['periodo'] < 3]) > 0:
                    print(tabela['data'][i])
                    print('entrada tarde != 13h', '\n')
                    sleep(3)##
                    for ind in dia_sel[dia_sel['periodo'] < 3].index.values: 
                        apagar_registro(ind)

    ##############################
    #quando o registro for do periodo tarde(3), com saida != 19h => apagar os registros posteriores (noite)
    condicao1 = tabela['periodo'] == 3
    condicao2 = tabela['Saida'] != '19:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]):
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            #se o plantão não for o unico no dia, devera ser analisado
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[tabela['data'] == tabela['data'][i]]
                #verificando se há plantão antes des deste
                if len(dia_sel[dia_sel['periodo'] > 3]) > 0:
                    print(tabela['data'][i])
                    print('saida tarde != 19h', '\n')
                    sleep(3)##
                    ind = dia_sel[dia_sel['periodo'] > 3].index.values[0]
                    apagar_registro(ind)

    ####################################################################
    #quando o registro for do periodo noite(4), com entrada != 19h => apagar os registros anteriores (manha, tarde)
    condicao1 = tabela['periodo'] == 4
    condicao2 = tabela['Entrada'] != '19:00'
    condicao3 = tabela['Dia Semana'] < 6
    if len(tabela[condicao1 & condicao2 & condicao3]):
        for i in tabela[condicao1 & condicao2 & condicao3].index.values:
            if len(tabela[tabela['data'] == tabela['data'][i]]) > 0: 
                dia_sel = tabela[tabela['data'] == tabela['data'][i]]
                if len(dia_sel[dia_sel['periodo'] < 4]) > 0:
                    print(tabela['data'][i])
                    print('entrada noite != 19', '\n')
                    sleep(3)##
                    count = 0
                    for ind in dia_sel[dia_sel['periodo'] < 4].index.values: 
                        apagar_registro(ind-count)
                        count += 1
        
    ###############################################################################################################
    #quando os periodos periodos estiverem duplicados => apagar duplicada num 1
    for data in tabela['data'].unique():
        dia_sel = tabela[tabela['data'] == data]
        if dia_sel['PERIODO'].duplicated().any():
            valores_duplicados = dia_sel['PERIODO'].duplicated()
            indices_duplicados = dia_sel[valores_duplicados].index
            count = 0
            print(data)
            print('plantões duplicados', '\n')
            break
            for i in dia_sel[valores_duplicados]['PERIODO'].values:
                ind = dia_sel[dia_sel['PERIODO'] == i].index.values[0]
                print(NOME, tabela['data'][ind], "apagado duplicata")
                sleep(3)##
                apagar_registro(ind-count)
                count += 1
                
    ####################################################################
    if count > 0:
        lista.append(count)
        
    count_control += 1
    
print(len(lista))
print(lista)
print(sum(lista))

In [ ]:
count_control += 1

# PREENCHENDO VAZIO

In [ ]:
df_total = pd.read_excel('Feitos.xlsx')
df_total.head(2)

In [ ]:
count_control = 0

In [ ]:
reset_selecao()
lista = []
count_nome = 0 

for NOME in NOMES[count_control:]:

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            count += 1
            
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
    
    tabela = funcao()
    tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
    
    count = 0
    for dia in tabela['Dia'].unique():
        tabela = funcao()
        tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
        dia_sel = tabela[tabela['Dia'] == dia].reset_index(drop=True)
        if len(dia_sel['Unidade'].unique()) > 1:
            print('alterando unidades, dia:', dia)

            while len(dia_sel['Unidade'].unique()) > 1:
                
                condicao = df_total['nome'] == unidecode.unidecode(NOME.upper())
                condicao1 = df_total['data'] == dia_sel['data'].values[0]
                unidade_do_dia = df_total['unidade'][condicao & condicao1]
                if len( unidade_do_dia ) == 0:
                    print('VERIFICAR'*5)
                    break
                unidade_do_dia = unidade_do_dia.values[0]
                dia_sel2 = dia_sel[dia_sel['Unidade'] != unidade_do_dia]
                
                print(dia_sel2['Unidade'].values[0], '=>', unidade_do_dia)
                #abrindo/selecionando periodo conforme localização
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel2['Semana'].values[0])+']/td['+str(dia_sel2['Dia Semana'].values[0])+']/ul['+str(dia_sel2['ordem'].values[0])+']/li'
                xpath_click(xpath_localizacao)
                
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(0.5)
                
                #Selecionando Setor
                periodozin = dia_sel2['Periodo'].values[0]
                periodozin = periodozin.replace('COBERTURA EXTRA','C.CIRURGICO') 
                periodozin = periodozin.replace('AV.PRE ANESTESICA','C.CIRURGICO')
                setor(unidade_do_dia, periodozin)

                if diferenca_horas(dia_sel2['Entrada'].values[0], dia_sel2['Saida'].values[0]) >= 5*60:
                    pontuacao(100)
                    pontuacao(100)
                    
                #salvar plantão
                sleep(1)
                atualizando_plantao()
                atualizando_plantao()
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
                sleep(5)

                tabela = funcao()
                tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
                dia_sel = tabela[tabela['Dia'] == dia].reset_index(drop=True)
                
    count_control += 1

In [ ]:
condicao1

In [ ]:
count_control += 1
print(NOMES[count_control])

In [ ]:
/html/body/div[10]/div/input

In [ ]:
dia_sel

In [ ]:
#Selecionando Setor
unidade_do_dia = dia_sel['Unidade'].value_counts().idxmax()

periodozin = dia_sel2['Periodo'].values[0]
periodozin = periodozin.replace('COBERTURA EXTRA','C.CIRURGICO') 
periodozin = periodozin.replace('AV.PRE ANESTESICA','C.CIRURGICO')
setor(unidade_do_dia, periodozin)

if diferenca_horas(dia_sel2['Entrada'].values[0], dia_sel2['Saida'].values[0]) >= 5*60:
    pontuacao(100)
    pontuacao(100)
    
#salvar plantão
sleep(1)
atualizando_plantao()
atualizando_plantao()
sleep(0.2)


count_control += 1

In [ ]:
print(unidade_do_dia)
print(dia_sel['Periodo'][i])

In [ ]:
scrap_unidades['momento'].unique()

In [ ]:
count_control += 1

In [ ]:
fdzão

# MATANDO O RESTO:

In [ ]:
fdzão = fdzão.rename({'Unidade':'unidade', 'Periodo':'setor',
                     'Entrada':'ENTRADA', 'Saida':'SAIDA'}, axis=1)

#DURAÇÃO DO PLANTÃO
fdzão['DURAÇÃO'] = np.nan

for i in range(0, len(fdzão)):
    diferenca_tempo = datetime.strptime(fdzão['SAIDA'][i], '%H:%M') - datetime.strptime(fdzão['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    fdzão['DURAÇÃO'][i] = horas + ':' + minutos
    
fdzão['DURAÇÃO'].unique()

fd = fdzão[['nome','data','unidade','ENTRADA','SAIDA','DURAÇÃO','setor']]

df = fdzão
df['extra'] = ''

df

In [ ]:
df['sem_ou_fds'] = "semana"

In [ ]:
H=0
print(len(df))

In [ ]:
df = df_prob.reset_index(drop=True)

In [ ]:
for h in range(H, len(df)):
    print(h)
    #click criar novo plantão
    browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
    #esta disponivel o botão de setor
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
    #Setor====
    print('base:', df['unidade'][h], '-', df['setor'][h])
    browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
    ind = scrap_unidades.loc[(scrap_unidades['unidade'] == df['unidade'][h]) & 
                             (scrap_unidades['momento'] == df['setor'][h])].index.values[0]
    unidade = scrap_unidades['unidade_count'][ind]
    setorzin = scrap_unidades['plt_count'][ind]
    print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
    #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
    print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
          browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))
    
    #listar todos
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()
    
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    sleep(0.5)
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(df['nome'][h])
    #Dando enter
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
    #info
    print('base:', df['nome'][h])
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
    print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)
    
    sleep(2)
    #Data
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(df['data'][h])
    sleep(0.5)
    print(df['data'][h])
    
    #Inicio do plantão
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys(df['ENTRADA'][h])
    sleep(1)
    print('hora de entrada:', df['ENTRADA'][h], '\nhora de saida:', df['SAIDA'][h])
    #Duração do plantão
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys(df['DURAÇÃO'][h])
    print('duração do plantão', df['DURAÇÃO'][h])
    sleep(2)
    
    ##TIPO
    '''
    //*[@id="select2-drop"]/ul/li[3]/div
    1 - Normal
    2 - DESLOCAMENTO
    3 - VOLUNTÁRIO
    4 - COBERTURA
    '''
    if df['extra'][h] == 'extra':
        print('EXTRA')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[3]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(200) #Pontos
            print('duração >5 -> 200 pontos')
        else:
            pass
    elif df['extra'][h] == 'DESLOCADO':
        print('DESLOCADO')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(2)
    else:
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
            
            sleep(1)
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
        else:
            pass
        sleep(2)
        
    #salvar plantão
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
        #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
        sleep(1)
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
        df.loc[int(h), 'controle'] = "PROBS"
        print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"
    
    #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

    #não esta disponivel o X
    
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    print('='*40, '\n')
    
    if H % 5 == 0:
        print('='*80)
        print(df['controle'].value_counts())
        print('='*80)
        
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df))
print(df['controle'].value_counts())

In [ ]:
H += 1

In [ ]:
#df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']]).reset_index(drop=True)
df_prob = df[df['controle'] != 'NORM'].reset_index(drop=True)
print(len(df_prob))
df = df[df['controle'] == 'NORM'].reset_index(drop=True)
print(len(df))

In [ ]:
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe.to_excel(nome_arquivo, index=False)
        print("+=", len(df1), '; len =', len(dataframe)) 

    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print("len =", len(dataframe)) 
        
salvando_nome_arquivo('Feitos_a+.xlsx', df)
print('.')
salvando_nome_arquivo('Não Feitos.xlsx', df_prob)

In [ ]:
df_prob

In [ ]:
browser.quit()